In [1]:
import os
import pandas as pd
import glob
import plotly.express as px
from pathlib import Path

In [2]:
dir_path = Path(os.getcwd())
data_dir = str(os.path.join(dir_path.parent,'data'))
balance_files = glob.glob(data_dir + "/poker_balances*.csv")
hands_files = glob.glob(data_dir + "/poker_hands*.csv")
table_files = glob.glob(data_dir + "/poker_table_info*.csv")

def get_hands_df():
    hands_dfs=[]

    for file in hands_files:
        new_df = pd.read_csv(file)
        hands_dfs.append(new_df)

    hands = pd.concat(hands_dfs)
    return hands
    
def get_games_df():
    balance_dfs=[]
    
    for file in balance_files:
        new_df = pd.read_csv(file)
        balance_dfs.append(new_df)
        
    balances = pd.concat(balance_dfs)
    return balances

def get_table_df():
    table_dfs=[]
    
    for file in table_files:
        new_df = pd.read_csv(file)
        table_dfs.append(new_df)
        
    tables = pd.concat(table_dfs)
    return tables


In [3]:
pd.set_option('display.max_columns', None)  
hands_base = get_hands_df()
games_base = get_games_df()
tables = get_table_df()

games = pd.merge(games_base,tables)
hands = pd.merge(hands_base,games)

In [4]:
tables

,table_id,scenario_name,player_types
0,1,smart vs 5 all different types player,AlwaysCallPlayer|AlwaysRaisePlayer|simpleLearn...
0,2,smart vs 5 all different types player,AlwaysCallPlayer|AlwaysRaisePlayer|simpleLearn...
0,3,smart vs 5 all different types player,AlwaysCallPlayer|AlwaysRaisePlayer|simpleLearn...


In [5]:
games

,table_id,game_id,player_name,player_type,game_result,game_reason,blind_type,final_hand,beginning_balance,game_start_balance,game_end_balance,game_net_change,scenario_name,player_types
0,1,1000001,players_3,simpleLearnerPlayer,lost,lost_game,big,two_pair,100000,100000.0,99990.0,-10.0,smart vs 5 all different types player,AlwaysCallPlayer|AlwaysRaisePlayer|simpleLearn...
1,1,1000002,players_3,simpleLearnerPlayer,lost,lost_game,None,one_pair,100000,99990.0,99990.0,0.0,smart vs 5 all different types player,AlwaysCallPlayer|AlwaysRaisePlayer|simpleLearn...
2,1,1000003,players_3,simpleLearnerPlayer,won,won_game,small,one_pair,100000,99990.0,100325.0,335.0,smart vs 5 all different types player,AlwaysCallPlayer|AlwaysRaisePlayer|simpleLearn...
3,1,1000004,players_3,simpleLearnerPlayer,lost,lost_game,big,two_pair,100000,100325.0,100315.0,-10.0,smart vs 5 all different types player,AlwaysCallPlayer|AlwaysRaisePlayer|simpleLearn...
4,1,1000005,players_3,simpleLearnerPlayer,lost,lost_game,None,one_pair,100000,100315.0,100315.0,0.0,smart vs 5 all different types player,AlwaysCallPlayer|AlwaysRaisePlayer|simpleLearn...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,3,3000006,players_2,AlwaysRaisePlayer,won,won_game,None,one_pair,100000,99150.0,99320.0,170.0,smart vs 5 all different types player,AlwaysCallPlayer|AlwaysRaisePlayer|simpleLearn...
86,3,3000007,players_2,AlwaysRaisePlayer,won,won_game,small,one_pair,100000,99320.0,99490.0,170.0,smart vs 5 all different types player,AlwaysCallPlayer|AlwaysRaisePlayer|simpleLearn...
87,3,3000008,players_2,AlwaysRaisePlayer,won,won_game,big,two_pair,100000,99490.0,99660.0,170.0,smart vs 5 all different types player,AlwaysCallPlayer|AlwaysRaisePlayer|simpleLearn...
88,3,3000009,players_2,AlwaysRaisePlayer,won,won_game,None,straight,100000,99660.0,99830.0,170.0,smart vs 5 all different types player,AlwaysCallPlayer|AlwaysRaisePlayer|simpleLearn...


In [10]:
px.line(games[games['table_id']==1],x='game_id',y='game_end_balance',color='player_name')

In [135]:
gain_loss_playerType=games.groupby(['table_id','player_type'])['game_net_change'].sum().reset_index()
gain_loss_playerType.groupby('player_type')['game_net_change'].mean().reset_index().sort_values(by='game_net_change')

,player_type,game_net_change
0,AlwaysCallPlayer,-26086.765156
1,AlwaysRaisePlayer,-23098.920711
2,CalculatedPlayer,-2699.278889
4,GambleByProbabilityPlayer,5285.166667
5,SmartPlayer,5300.398311
3,ConservativePlayer,41299.399778


In [148]:
games[games['game_result']=='won'].groupby(['player_type','game_result'])['blind_type'].count().reset_index().sort_values(by='blind_type')

,player_type,game_result,blind_type
3,ConservativePlayer,won,34
2,CalculatedPlayer,won,263
4,GambleByProbabilityPlayer,won,263
5,SmartPlayer,won,610
1,AlwaysRaisePlayer,won,978
0,AlwaysCallPlayer,won,995


In [6]:
hands

,table_id,game_id,player_name,player_type,bet_number,opponents,call,current,final,pot,allowed,hand1,hand2,community1,community2,community3,community4,community5,game_result,game_reason,blind_type,beginning_balance,game_start_balance,game_end_balance,game_net_change,scenario_name,player_types
0,1,1,players_3,AlwaysCallPlayer,1,5,10,0,10.0,35,True,9-clubs,K-hearts,Z-N/A,Z-N/A,Z-N/A,Z-N/A,Z-N/A,won,won_game,None,100000,100000,100000,0,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
1,1,1,players_3,AlwaysCallPlayer,2,5,10,10,20.0,90,True,9-clubs,K-hearts,Z-N/A,Z-N/A,Z-N/A,Z-N/A,Z-N/A,won,won_game,None,100000,100000,100000,0,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
2,1,1,players_3,AlwaysCallPlayer,3,5,10,20,30.0,150,False,9-clubs,K-hearts,Z-N/A,Z-N/A,Z-N/A,Z-N/A,Z-N/A,won,won_game,None,100000,100000,100000,0,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
3,1,1,players_3,AlwaysCallPlayer,4,5,0,30,30.0,180,True,9-clubs,K-hearts,3-clubs,8-spades,J-diamonds,Z-N/A,Z-N/A,won,won_game,None,100000,100000,100000,0,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
4,1,1,players_3,AlwaysCallPlayer,5,5,10,30,40.0,210,True,9-clubs,K-hearts,3-clubs,4-clubs,8-spades,J-diamonds,Z-N/A,won,won_game,None,100000,100000,100000,0,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203548,9,900,players_2,AlwaysCallPlayer,1,5,5,5,10.0,55,True,10-spades,6-diamonds,Z-N/A,Z-N/A,Z-N/A,Z-N/A,Z-N/A,won,won_game,small,100000,100050,100050,0,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
203549,9,900,players_2,AlwaysCallPlayer,2,5,0,10,10.0,60,True,10-spades,6-diamonds,4-clubs,7-diamonds,Q-hearts,Z-N/A,Z-N/A,won,won_game,small,100000,100050,100050,0,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
203550,9,900,players_2,AlwaysCallPlayer,3,5,0,10,10.0,60,True,10-spades,6-diamonds,4-clubs,7-clubs,7-diamonds,Q-hearts,Z-N/A,won,won_game,small,100000,100050,100050,0,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
203551,9,900,players_2,AlwaysCallPlayer,4,5,10,10,20.0,80,True,10-spades,6-diamonds,4-clubs,7-clubs,7-diamonds,Q-hearts,Z-N/A,won,won_game,small,100000,100050,100050,0,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...


# Basic Game Statistics

Number of Games played in the simulation

In [7]:
games = games[games['scenario_name'] == 'all_raise_against_smart_player']

In [8]:
number_of_games = len(set(games.game_id))
number_of_games

2500

Players Mean number of chips gained and variance

In [9]:
chips_gained_per_game = games.groupby('player_name').mean('game_net_change')[['game_net_change']].reset_index()
variance_calculation = games[['player_name','game_net_change']]
variance_calculation['game_net_change'] = variance_calculation['game_net_change'] ** 2
expectation_squared = variance_calculation.groupby('player_name').mean('game_net_change').reset_index()
merged_expectations = pd.merge(expectation_squared,chips_gained_per_game,on='player_name')
merged_expectations['variance'] = merged_expectations['game_net_change_x'] - merged_expectations['game_net_change_y']
summary_stats = merged_expectations[['player_name','game_net_change_y','variance']]
summary_stats.columns = ['player_name','mean','variance']
summary_stats['games'] = number_of_games
summary_stats

c:\users\silbe\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,player_name,mean,variance,games
0,players_1,0.5108,3.5088,2500
1,players_2,0.5108,3.5088,2500
2,players_3,0.5108,3.5088,2500
3,players_4,0.5108,3.5088,2500
4,players_5,0.5108,3.5088,2500
5,players_6,-2.5540,103.0440,2500


Top 10 Best Card Hands Dealt by Mean Number of Chips Gained 

In [10]:
two_card_hands = hands[['game_id','player_name','player_type','hand1','hand2','game_net_change']]
two_card_hands = two_card_hands.drop_duplicates()
best_two_card_hands = two_card_hands.groupby(['player_name','hand1','hand2']).mean().reset_index()[['player_name','hand1','hand2','game_net_change']]
best_two_card_hands = best_two_card_hands.sort_values(by=['player_name','game_net_change'], ascending=[True,False])
best_two_card_hands.groupby('player_name').head(10)

,player_name,hand1,hand2,game_net_change
830,players_1,6-clubs,J-clubs,12.000000
186,players_1,10-spades,K-hearts,10.000000
356,players_1,2-spades,9-spades,9.000000
1095,players_1,8-spades,9-hearts,8.666667
1094,players_1,8-spades,9-diamonds,8.000000
1234,players_1,J-clubs,K-clubs,8.000000
401,players_1,3-clubs,A-hearts,7.000000
520,players_1,3-spades,9-spades,7.000000
120,players_1,10-hearts,7-diamonds,6.000000
299,players_1,2-hearts,6-hearts,6.000000


In [17]:
import numpy as np

a=np.array([3,3,-30])
b=np.array([4,5,6])
a+b

array([  7,   8, -24])

In [21]:
import sys
print(sys.executable)

C:\Users\shiras.PHILLIPS66\AppData\Local\Continuum\anaconda3\python.exe


In [22]:
np.__file__

'C:\\Users\\shiras.PHILLIPS66\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\numpy\\__init__.py'